In [1]:
library(tidyverse)
library(RPostgreSQL)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DBI


In [2]:
sql_dir <- file.path("..", "SQL")
list.files(sql_dir)

[1] "basics.sql"              "cohort.sql"             
 [3] "drugs.sql"               "fluid_input.sql"        
 [5] "fluid_output.sql"        "fluid.sql"              
 [7] "icd9.sql"                "lab_tests.sql"          
 [9] "lab_unpivot.sql"         "merge_data.sql"         
[11] "population.sql"          "second_day"             
[13] "serum_diff.sql"          "serum_echo_diff.sql"    
[15] "serum_non_diff.sql"      "subgroup.sql"           
[17] "third-day"               "tte-summary.sql"        
[19] "vital_signs_unpivot.sql" "vital_signs.sql"

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

#### Angus cohort

In [4]:
dbGetQuery(con, "select count(*) from population where angus = 1;")

count
17420


#### Adult in their 1st ICU stay

In [5]:
sql <- "
select count(*)
  from population
 where angus = 1
   and age >= 18
   and icu_order = 1
"
dbGetQuery(con, sql)

count
10566


#### MICU/SICU only

In [6]:
sql <- "
select count(*)
  from population
 where angus = 1
   and age >= 18
   and icu_order = 1
   and first_careunit in ('MICU', 'SICU')
"
dbGetQuery(con, sql)

count
7184


In [7]:
sql <- "
select count(*)
  from population
 where angus = 1
   and age >= 18
   and icu_order = 1
   and first_careunit in ('MICU', 'SICU')
   and echo_exclude = 0
"
dbGetQuery(con, sql)

count
6362


In [8]:
sql <- "
select count(*)
  from population
 where angus = 1
   and age >= 18
   and icu_order = 1
   and first_careunit in ('MICU', 'SICU')
   and echo_include = 1
"
dbGetQuery(con, sql)

count
6361


In [9]:
sql <- "
select count(*)
  from population
 where angus = 1
   and age >= 18
   and icu_order = 1
   and first_careunit in ('MICU', 'SICU')
   and echo_time is not null
   and echo_include = 1
"
dbGetQuery(con, sql)

count
3262


In [10]:
# dbDisconnect(con)
# dbUnloadDriver(drv)